In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
vendor_name = 'SHAH KOTHARI BROS'
start_date_pn = '01-09-2024'
end_date_pn = '31-12-2024'

start_date_cn = '01-09-2024'
end_date_cn = '23-01-2025'

In [3]:
pn = pd.read_csv('pn_data.csv')
pn.head()

,vendor_id,vendor_name,pick_note_number,date,key,ws_code,batch_number,quantity,month
0,342,J.R.SHAH AND COMPANY,PR201008001,01-10-2024,342_19316_O309,19316,O309,1,Oct-2024
1,356,ATUL MEDICAL AGENCY,PR201008005,01-10-2024,356_4382_K2400513,4382,K2400513,1,Oct-2024
2,357,DHRUVI HEALTHCARE PVT LTD,PR201008007,01-10-2024,357_1091_BASM036,1091,BASM036,1,Oct-2024
3,358,B K DISTRIBUTORS,PR201007997,01-10-2024,358_23588_2C29L009,23588,2C29L009,85,Oct-2024
4,359,D K PHARMA,PR201008003,01-10-2024,359_5457_KGA2301A,5457,KGA2301A,1,Oct-2024


In [4]:
cn = pd.read_csv('cn_data.csv')
cn.head()

,vendor_cn_date,debit_note_number,key,billed_quantity,month,vendor_name
0,2024-09-01,DN201005468,388_10098_GEO37697,1,Sep-2024,PHARMA DISTRIBUTOR
1,2024-09-01,DN201005466,388_10779_LIL1L00,20,Sep-2024,PHARMA DISTRIBUTOR
2,2024-09-01,DN201005468,388_10779_LIL1L00,5,Sep-2024,PHARMA DISTRIBUTOR
3,2024-09-01,DN201005468,388_11141_230330040,1,Sep-2024,PHARMA DISTRIBUTOR
4,2024-09-01,DN201005468,388_11681_220910719,3,Sep-2024,PHARMA DISTRIBUTOR


In [5]:
pn['date'] = pd.to_datetime(pn['date'], format='%d-%m-%Y')
cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'])

In [6]:
# 3. Sort DataFrames
pn = pn.sort_values('date')
cn = cn.sort_values('vendor_cn_date')

In [12]:
import pandas as pd
from datetime import datetime

def map_pn_cn():
    # Load data
    pn = pd.read_csv('pn_data.csv')
    cn = pd.read_csv('cn_data.csv')
    
    # Convert dates
    pn['date'] = pd.to_datetime(pn['date'], format='%d-%m-%Y')
    cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'])
    
    # Sort by date
    pn = pn.sort_values('date')
    cn = cn.sort_values('vendor_cn_date')
    
    mapped_records = []
    remaining_pn = {}  # {key: [(date, qty_remaining),...]}
    
    # Group by key
    for key in pn['key'].unique():
        key_pns = pn[pn['key'] == key].copy()
        key_cns = cn[cn['key'] == key].copy()
        
        if key_cns.empty:
            continue
            
        pn_qty_tracker = []  # Track PN quantities and their mapping status
        for _, pn_row in key_pns.iterrows():
            pn_qty_tracker.append({
                'date': pn_row['date'],
                'qty': pn_row['quantity'],
                'remaining': pn_row['quantity'],
                'pn_data': pn_row
            })
        
        # Process each CN
        for _, cn_row in key_cns.iterrows():
            cn_qty_remaining = cn_row['billed_quantity']
            
            # Find eligible PNs (dates before or equal to CN date)
            eligible_pns = [p for p in pn_qty_tracker 
                          if p['remaining'] > 0 and 
                          p['date'] <= cn_row['vendor_cn_date']]
            
            while cn_qty_remaining > 0 and eligible_pns:
                # Take the nearest date PN first
                current_pn = min(eligible_pns, 
                               key=lambda x: abs((cn_row['vendor_cn_date'] - x['date']).days))
                
                # Calculate quantity to map
                qty_to_map = min(current_pn['remaining'], cn_qty_remaining)
                
                mapped_records.append({
                    'PN_Date': current_pn['date'],
                    'CN_Date': cn_row['vendor_cn_date'],
                    'Pick_Note': current_pn['pn_data']['pick_note_number'],
                    'Credit_Note': cn_row['debit_note_number'],
                    'Key': key,
                    'Vendor': current_pn['pn_data']['vendor_name'],
                    'PN_Qty': qty_to_map,
                    'CN_Qty': qty_to_map,
                    'WS_Code': current_pn['pn_data']['ws_code'],
                    'Batch': current_pn['pn_data']['batch_number'],
                    'Original_PN_Qty': current_pn['pn_data']['quantity'],
                    'Original_CN_Qty': cn_row['billed_quantity']
                })
                
                current_pn['remaining'] -= qty_to_map
                cn_qty_remaining -= qty_to_map
                
                if current_pn['remaining'] == 0:
                    eligible_pns.remove(current_pn)
    
    # Create final DataFrame
    result_df = pd.DataFrame(mapped_records)
    
    # Format dates
    result_df['PN_Date'] = result_df['PN_Date'].dt.strftime('%Y-%m-%d')
    result_df['CN_Date'] = result_df['CN_Date'].dt.strftime('%Y-%m-%d')
    
    # Sort results
    result_df = result_df.sort_values(['Key', 'PN_Date', 'CN_Date'])
    
    # Save to Excel
    result_df.to_csv('PN_CN_Mapping_Results.csv', index=False)
    return result_df

if __name__ == "__main__":
    results = map_pn_cn()
    print("Mapping completed. Check PN_CN_Mapping_Results.csv")

Mapping completed. Check PN_CN_Mapping_Results.csv


In [1]:
import pandas as pd
from datetime import datetime
from collections import defaultdict

def process_mapping():
    # Load and prepare data
    pn = pd.read_csv('pn_data.csv')
    cn = pd.read_csv('cn_data.csv')
    
    pn['date'] = pd.to_datetime(pn['date'], format='%d-%m-%Y')
    cn['vendor_cn_date'] = pd.to_datetime(cn['vendor_cn_date'])
    
    pn = pn.sort_values('date')
    cn = cn.sort_values('vendor_cn_date')
    
    mapped_records = []
    cn_remaining = defaultdict(dict)  # Track remaining CN quantities by key and date
    
    # Process each key
    for key in pn['key'].unique():
        key_pns = pn[pn['key'] == key].copy()
        key_cns = cn[cn['key'] == key].copy()
        
        if key_cns.empty:
            continue
            
        # Initialize CN remainings for this key
        for _, cn_row in key_cns.iterrows():
            cn_remaining[key][cn_row['vendor_cn_date']] = cn_row['billed_quantity']
            
        pn_remaining = 0  # Track remaining PN quantity
        
        for _, pn_row in key_pns.iterrows():
            current_pn_qty = pn_row['quantity'] + pn_remaining
            
            # Find eligible CNs with remaining quantity
            eligible_cns = key_cns[
                (key_cns['vendor_cn_date'] >= pn_row['date']) & 
                (key_cns['vendor_cn_date'].map(lambda x: cn_remaining[key][x] > 0))
            ].copy()
            
            if eligible_cns.empty:
                pn_remaining = current_pn_qty
                continue
                
            nearest_cn = eligible_cns.iloc[0]
            cn_date = nearest_cn['vendor_cn_date']
            cn_qty_available = cn_remaining[key][cn_date]
            
            qty_to_map = min(current_pn_qty, cn_qty_available)
            
            mapped_records.append({
                'PN_Date': pn_row['date'],
                'CN_Date': cn_date,
                'Pick_Note': pn_row['pick_note_number'],
                'Credit_Note': nearest_cn['debit_note_number'],
                'Key': key,
                'Vendor': pn_row['vendor_name'],
                'PN_Qty': qty_to_map,
                'CN_Qty': qty_to_map,
                'PN_Remaining': current_pn_qty - qty_to_map,
                'CN_Remaining': cn_qty_available - qty_to_map,
                'Original_PN_Qty': pn_row['quantity'],
                'Original_CN_Qty': nearest_cn['billed_quantity']
            })
            
            # Update remaining quantities
            cn_remaining[key][cn_date] -= qty_to_map
            pn_remaining = current_pn_qty - qty_to_map
            
    result_df = pd.DataFrame(mapped_records)
    result_df['PN_Date'] = result_df['PN_Date'].dt.strftime('%Y-%m-%d')
    result_df['CN_Date'] = result_df['CN_Date'].dt.strftime('%Y-%m-%d')
    
    result_df.to_csv('p_c_r.csv', index=False)
    return result_df

if __name__ == "__main__":
    results = process_mapping()
    print("Mapping completed. Check p_c_r.csv")

Mapping completed. Check p_c_r.csv
